In [9]:
from selenium import webdriver

from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait as Wait
from selenium.webdriver.support import expected_conditions as EC

from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException

from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains

from bs4 import BeautifulSoup as bs

from time import sleep

from webdriver_manager.chrome import ChromeDriverManager

import csv

import requests

from lxml import html

import re

In [10]:
with open('doc_links2019.txt', 'w') as l:
    pass

In [22]:
# НАЖИМАЕМ КНОПКУ "ЗАГРУЗИТЬ ЕЩЕ" 

def click_more(driver):
    span = '//*[@id="searchResults"]/div[2]/span'
    element = Wait(driver, 15).until(EC.presence_of_element_located((By.XPATH, span)))
    try:
        element.click()
    except:
        pass
    
    
def down_more(driver, soup):
    articles = soup.find('div', class_='b-news-inner__list')
    divs = articles.find_all('div', class_='b-news-inner__list-item b-news-inner__list-item_doc')
    
    for raz in range(20):
        try:
            last = divs[len(divs)-1].find('div', class_='b-news-inner__list-item-date _date').get_text()
            if bool(re.match('01.\d\d.\d\d\d\d', str(last))) is True:
                print(raz, 'Nothing to search here')
                break

            try:
                click_more(driver)

            except ElementClickInterceptedException:
                action = ActionChains(driver)
                action.key_down(Keys.END).perform()
                sleep(3)
                click_more(driver)

            except:
                print(raz, "cannot search more")
                break    
        
        except IndexError:
            pass

In [12]:
# СОБИАРЕМ ССЫЛКИ (ФУНКЦИЯ)

def gather_links(driver, links, soup):
    host = 'https://rg.ru'
    for article in soup.findAll('div', class_='b-news-inner__list-item-wrapper'):
        link = article.find('a', href=True)
        wholelink = host+link['href']
        links.write(wholelink)
        links.write('\n')

In [13]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.maximize_window()



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [C:\Users\Марк\.wdm\drivers\chromedriver\win32\99.0.4844.51\chromedriver.exe] found in cache
<ipython-input-13-0d9242eb10b9>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [28]:
# ИЩЕМ ССЫЛКИ

with open('doc_links2019.txt', 'a') as links:
    for year in range(2018, 2017, -1):
        for month in range(8, 0, -1):
            
            month = str(month).zfill(2)
            print(f'00.{month}.{year}')
            
            for d in range(4):
                if d == 0:
                    first_day = '24'
                    
                    if month == '02':
                        last_day = '28'
                    elif month == '04' or '06' or '09' or '11':
                        last_day = '30'
                    else:
                        last_day = '31'
                        
                elif d == 1:
                    first_day = '16'
                    last_day = '23'
                    
                elif d == 2:
                    first_day = '8'
                    last_day = '15'
                
                else:
                    first_day = '1'
                    last_day = '7'
                    
                first_day = str(first_day).zfill(2)
                last_day = str(last_day).zfill(2)
                
                
                try:
                    driver.get(f'https://rg.ru/doc-search/?materialTypes=doc&from={first_day}.{month}.{year}&to={last_day}.{month}.{year}')
                    soup = bs(driver.page_source, 'html.parser')
                    nothing = soup.find('div', class_='b-news-inner__list').get_text()
                    nichego = 'Ничего не найдено'
                    if nichego in nothing:
                        continue

                    else:
                        down_more(driver, soup)
                        gather_links(driver, links, soup)
                        
                        
                except WebDriverException:
                    pass
                

00.08.2018
00.07.2018
00.06.2018
00.05.2018
00.04.2018
00.03.2018
00.02.2018
00.01.2018


In [168]:
soup = bs(driver.page_source, 'html.parser')
nothing = soup.find('div', class_='b-news-inner__list').get_text()
print(nothing)

  Ничего не найдено


0
